## Neural Network Regression
In this example, we will create a neural network mlpack for regression.

### 1. Setup
Include all libraries required to implement this tutorial. These mainly include files from mlpack, ensmallen and armadillo.

In [1]:
#include <mlpack/core.hpp>
#include <mlpack/methods/ann/loss_functions/mean_squared_error.hpp>
#include <mlpack/core/data/scaler_methods/min_max_scaler.hpp>
#include <mlpack/methods/ann/layer/layer.hpp>
#include <mlpack/core/data/split_data.hpp>
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/ann/init_rules/he_init.hpp>
#include <ensmallen.hpp>

Some convenient namespaces to simplify the tutorial.

In [2]:
using namespace mlpack;

In [3]:
using namespace mlpack::ann;

In [4]:
using namespace ens;

### 2. Set Model and Training parameters.
Set the training parameters for the model.

In [5]:
// Testing data is taken from the dataset in this ratio.
constexpr double RATIO = 0.1; //10%

//! - H1: The number of neurons in the 1st layer.
constexpr int H1 = 64;
//! - H2: The number of neurons in the 2nd layer.
constexpr int H2 = 128;
//! - H3: The number of neurons in the 3rd layer.
constexpr int H3 = 64;

// Number of epochs for training. Increase number of epochs for better results.
const int EPOCHS = 30;
//! - STEP_SIZE: Step size of the optimizer.
constexpr double STEP_SIZE = 5e-2;
//! - BATCH_SIZE: Number of data points in each iteration of SGD.
constexpr int BATCH_SIZE = 32;
//! - STOP_TOLERANCE: Stop tolerance;
// A very small number implies that we do all iterations.
constexpr double STOP_TOLERANCE = 1e-8;

Set paths for the dataset, trained model and final predictions.

In [6]:
//! Path to the dataset used for training and testing.
const std::string datasetPath = "./bodyfat.tsv";
// File for saving the model.
const std::string modelFile = "nn_regressor.bin";

### 3. Loading the Dataset.
The bodyfat dataset contains estimates of the percentage of body fat determined by
underwater weighing and various body circumference measurements for 252 men. Accurate measurement of body fat is very expensive,but by using machine learning it is possible to calculate a prediction with good accuracy by just using some low cost
measurements of the body. The columns in the dataset are the following:

* Percent body fat (%) => this is the decision column (what we want to get from the model).
* Age (years)
* Weight (lbs)
* Height (inches)
* Neck circumference (cm)
* Chest circumference (cm)
* Abdomen 2 circumference (cm)
* Hip circumference (cm)
* Thigh circumference (cm)
* Knee circumference (cm)
* Ankle circumference (cm)
* Biceps (extended) circumference (cm)
* Forearm circumference (cm)
* Wrist circumference (cm)
* Density determined from underwater weighing

In [7]:
arma::mat dataset;
// In Armadillo columns represent data points, rows represent features.
data::Load(datasetPath, dataset, true);

### 4. Preprocess the dataset.
Split the data into training and validation set. We will also scale the data to increase stability in training.

In [8]:
// Split the dataset into training and validation sets.
arma::mat trainData, validData;
data::Split(dataset, trainData, validData, RATIO);

Split the data into input features and predictions.

In [9]:
// The train and valid datasets contain both - the features as well as the
// prediction. Split these into separate matrices.
arma::mat trainX = trainData.submat(1, 0, trainData.n_rows - 1,
                                    trainData.n_cols - 1);
arma::mat validX = validData.submat(1, 0, validData.n_rows - 1,
                                    validData.n_cols - 1);

// Create prediction data for training and validatiion datasets.
arma::mat trainY = trainData.row(0);
arma::mat validY = validData.row(0);

// Scale all data into the range (0, 1) for increased numerical stability.
data::MinMaxScaler scaleX;
// Scaler for predictions.
data::MinMaxScaler scaleY;

// Fit scaler only on training data.
scaleX.Fit(trainX);
scaleX.Transform(trainX, trainX);
scaleX.Transform(validX, validX);

// Scale training predictions.
scaleY.Fit(trainY);
scaleY.Transform(trainY, trainY);
scaleY.Transform(validY, validY);

### 5. Create the Model
Specifying the neural network model. The model consists of 3 hidden linear layers and uses LeakyReLU as activation function.

In [10]:
// This intermediate layer is needed for connection between input
// data and the next LeakyReLU layer.
// Parameters specify the number of input features and number of
// neurons in the next layer.
FFN<MeanSquaredError<>, HeInitialization> model;
model.Add<Linear<>>(trainX.n_rows, H1);
// Activation layer:
model.Add<LeakyReLU<>>();
// Connection layer between two activation layers.
model.Add<Linear<>>(H1, H2);
// Activation layer.
model.Add<LeakyReLU<>>();
// Connection layer.
model.Add<Linear<>>(H2, H3);
// Activation layer.
model.Add<LeakyReLU<>>();
// Connection layer => output.
// The output of one neuron is the regression output for one record.
model.Add<Linear<>>(H3, 1);

### 6. Training the model.
We will use ensmallen to create the optimizer and train the model. For more details refer to the [documentation](https://www.ensmallen.org/docs.html).

In [11]:
ens::Adam optimizer(
    STEP_SIZE, // Step size of the optimizer.
    BATCH_SIZE, // Batch size. Number of data points that are used in each iteration.
    0.9, // Exponential decay rate for the first moment estimates.
    0.999, // Exponential decay rate for the weighted infinity norm estimates.
    1e-8, // Value used to initialise the mean squared gradient parameter.
    trainData.n_cols * EPOCHS, // Max number of iterations.
    1e-8,// Tolerance.
    true);

Here we will use ensmallen callbacks to train the model. We will be using Adam optimizer. To stop the training when the loss stops decreasing or doesn't show any improvement.

In [12]:
// Train the model.
model.Train(trainX,
            trainY,
            optimizer,
            // PrintLoss Callback prints loss for each epoch.
            ens::PrintLoss(),
            // Progressbar Callback prints progress bar for each epoch.
            // Here 40 signifies width of progress bar.
            ens::ProgressBar(40),
            // Stops the optimization process if the loss stops decreasing
            // or no improvement has been made. This will terminate the
            // optimization once we obtain a minima on training set.
            ens::EarlyStopAtMinLoss(20));

Epoch 1/852
0.179052
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.179052
Epoch 2/852
0.00513052
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.00513052
Epoch 3/852
0.00688234
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.00688234
Epoch 4/852
0.00184822
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.00184822
Epoch 5/852
0.00105378
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.00105378
Epoch 6/852
0.000750528
8/8 [==================================================] 100% - 0s 7ms/step - loss: 0.000750528
Epoch 7/852
0.000693577
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.000693577
Epoch 8/852
0.000746652
8/8 [==================================================] 100% - 0s 1ms/step - loss: 0.000746652
Epoch 9/852
0.00066746
8/8 [==========================

### 7. Running Inference
Get predictions on validation dataset and test the quality of our model by calculating Mean Squared Error on it.

In [13]:
// Create predictions on the dataset.
arma::mat predOut;
model.Predict(validX, predOut);

// Calculate MSE loss on predictions.
double validMSE = metric::SquaredEuclideanDistance::Evaluate(predOut, validY) / (validY.n_elem);
std::cout << "Mean Squared Error on Prediction data points: " << validMSE << std::endl;

// Get meanigful prediction by inverse - scaling on predictions.
scaleY.InverseTransform(predOut, predOut);

Mean Squared Error on Prediction data points: 0.0087528


### 9. Loading and Saving Models
In the real world, we won't be training the model from scratch everytime we need to run inference.
We will save the model once and load it as many times as we want for either training or inference.

In [ ]:
data::Save(modelFile, "NNRegressor", model);
FFN<MeanSquaredError<>, HeInitialization> modelP;
// Load weights into the model.
data::Load(modelFile, "NNRegressor", modelP);